# Part 4: What We Found - Task Types Have Distinct Signatures

**Series**: Latent Diagnostics Analysis (4 of 5)

This notebook presents the core positive results: different task types produce measurably different activation patterns, and these differences persist after controlling for text length.

---

## Table of Contents

1. [Load and Prepare Data](#1-load-and-prepare-data)
2. [The Five Task Types](#2-the-five-task-types)
3. [Raw vs Length-Controlled Comparison](#3-raw-vs-length-controlled-comparison)
4. [The Gradient from Focused to Diffuse](#4-the-gradient-from-focused-to-diffuse)
5. [Statistical Validation](#5-statistical-validation)
6. [PCA Shows Natural Clustering](#6-pca-shows-natural-clustering)

In [ ]:
# Standard imports
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import linregress
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from IPython.display import Image, display

# Paths
DATA_DIR = Path('../data/results')
FIGURES_DIR = Path('../figures/paper')

# Style
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100

# Colors
COLORS = {
    'cola': '#2ecc71',
    'winogrande': '#3498db',
    'snli': '#9b59b6',
    'hellaswag': '#e74c3c',
    'paws': '#e67e22',
}

LABELS = {
    'cola': 'CoLA (Grammar)',
    'winogrande': 'WinoGrande (Commonsense)',
    'snli': 'SNLI (Inference)',
    'hellaswag': 'HellaSwag (Completion)',
    'paws': 'PAWS (Paraphrase)',
}

print("Environment ready.")

---

## 1. Load and Prepare Data

In [ ]:
# Load domain attribution metrics
with open(DATA_DIR / 'domain_attribution_metrics.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['samples'])
df['text_length'] = df['text'].str.len()

print(f"Loaded {len(df)} samples")
print(f"\nSamples per task type:")
print(df['source'].value_counts())

In [ ]:
# Helper functions
def residualize(metric, length):
    """Remove length effect from metric."""
    slope, intercept, _, _, _ = linregress(length, metric)
    return metric - (slope * length + intercept)

def cohens_d(a, b):
    """Cohen's d effect size."""
    na, nb = len(a), len(b)
    pooled_std = np.sqrt(((na-1)*np.std(a, ddof=1)**2 + (nb-1)*np.std(b, ddof=1)**2) / (na+nb-2))
    if pooled_std == 0:
        return 0
    return (np.mean(a) - np.mean(b)) / pooled_std

# Add residualized metrics
metrics = ['n_active', 'mean_influence', 'top_100_concentration', 'mean_activation']
for m in metrics:
    df[f'{m}_resid'] = residualize(df[m].values, df['text_length'].values)

print("Added length-controlled metrics.")

---

## 2. The Five Task Types

We analyze samples from five NLP benchmark tasks, each requiring different cognitive operations:

| Dataset | Task Type | What It Tests |
|---------|-----------|---------------|
| **CoLA** | Grammar | Linguistic acceptability ("Is this sentence grammatical?") |
| **WinoGrande** | Commonsense | Pronoun resolution requiring world knowledge |
| **SNLI** | Inference | Natural language inference (entailment, contradiction, neutral) |
| **HellaSwag** | Completion | Sentence completion requiring situational understanding |
| **PAWS** | Paraphrase | Semantic similarity detection |

In [ ]:
# Example from each task type
print("Example samples:")
print("=" * 70)
for src in ['cola', 'winogrande', 'snli', 'hellaswag', 'paws']:
    example = df[df['source'] == src].iloc[0]['text']
    print(f"\n{LABELS[src]}:")
    print(f"  \"{example[:80]}...\"" if len(example) > 80 else f"  \"{example}\"")

In [ ]:
# Summary statistics by task type
summary = df.groupby('source').agg({
    'text_length': 'mean',
    'n_active': 'mean',
    'mean_influence': 'mean',
    'top_100_concentration': 'mean'
}).round(4)

summary.columns = ['Avg Length', 'Avg N Active', 'Avg Influence', 'Avg Concentration']
print("Raw metrics by task type:")
summary

---

## 3. Raw vs Length-Controlled Comparison

This is the critical comparison. Which signals are genuine, and which are artifacts of text length?

In [ ]:
# Display the length control comparison figure
fig_path = FIGURES_DIR / 'length_control_comparison.png'
if fig_path.exists():
    display(Image(filename=str(fig_path), width=800))
else:
    print(f"Figure not found: {fig_path}")

In [ ]:
# Compute effect sizes: Grammar vs Reasoning
cola = df[df['source'] == 'cola']
others = df[df['source'] != 'cola']

print("Effect Sizes: Grammar (CoLA) vs Reasoning Tasks")
print("=" * 70)
print(f"{'Metric':<25} {'Raw d':>12} {'Controlled d':>15} {'Change':>15}")
print("-" * 70)

for m in metrics:
    d_raw = cohens_d(cola[m].values, others[m].values)
    d_resid = cohens_d(cola[f'{m}_resid'].values, others[f'{m}_resid'].values)
    
    if abs(d_resid) < 0.2 and abs(d_raw) > 0.8:
        change = "COLLAPSED"
    elif abs(d_resid) > 0.5:
        change = "PERSISTS"
    else:
        change = ""
    
    print(f"{m:<25} {d_raw:>+12.2f} {d_resid:>+15.2f} {change:>15}")

print("-" * 70)

### The Key Finding

**Before length control:**
- n_active: d = 2.17 (HUGE difference)
- mean_influence: d = 3.22 (HUGE difference)

**After length control:**
- n_active: d = 0.07 (COLLAPSED - was entirely length-driven)
- mean_influence: d = 1.08 (PERSISTS - genuine signal)

**Conclusion**: The real signal is in influence and concentration, not feature counts.

---

## 4. The Gradient from Focused to Diffuse

Tasks don't just cluster into "grammar" vs "everything else." There's a gradient.

In [ ]:
# Display domain distributions figure
fig_path = FIGURES_DIR / 'domain_distributions.png'
if fig_path.exists():
    display(Image(filename=str(fig_path), width=800))
else:
    print(f"Figure not found: {fig_path}")

In [ ]:
# Rank tasks by mean influence (length-controlled)
task_profiles = df.groupby('source').agg({
    'mean_influence_resid': 'mean',
    'top_100_concentration_resid': 'mean'
}).round(4)

task_profiles = task_profiles.sort_values('mean_influence_resid', ascending=False)
task_profiles.columns = ['Influence (resid)', 'Concentration (resid)']

print("Task types ranked by influence (length-controlled):")
print("=" * 50)
for src in task_profiles.index:
    inf = task_profiles.loc[src, 'Influence (resid)']
    conc = task_profiles.loc[src, 'Concentration (resid)']
    print(f"  {LABELS[src]:<30} Inf: {inf:+.4f}  Conc: {conc:+.4f}")

### Interpretation

**High influence + High concentration = Focused computation**
- CoLA (grammar): Checking rules, few key features, strong connections

**Low influence + Low concentration = Diffuse computation**
- PAWS (paraphrase): Comparing meanings, many features, weak connections
- HellaSwag (completion): Integrating context, distributed processing

This matches intuition: grammar checking is "focused" (apply specific rules), while reasoning is "diffuse" (integrate many pieces of information).

---

## 5. Statistical Validation

Are these differences real, or could they be due to chance?

In [ ]:
# Display significance boxplots
fig_path = FIGURES_DIR / 'boxplots_significance.png'
if fig_path.exists():
    display(Image(filename=str(fig_path), width=800))
else:
    print(f"Figure not found: {fig_path}")

In [ ]:
# Bootstrap confidence intervals
def bootstrap_ci(group1, group2, n_boot=5000, seed=42):
    np.random.seed(seed)
    boot_ds = []
    for _ in range(n_boot):
        s1 = np.random.choice(group1, size=len(group1), replace=True)
        s2 = np.random.choice(group2, size=len(group2), replace=True)
        boot_ds.append(cohens_d(s1, s2))
    return np.percentile(boot_ds, 2.5), np.percentile(boot_ds, 97.5)

print("Bootstrap 95% Confidence Intervals (5000 resamples)")
print("=" * 60)
print(f"{'Metric':<25} {'d':>10} {'95% CI':>20}")
print("-" * 60)

for m in ['mean_influence_resid', 'top_100_concentration_resid', 'mean_activation_resid']:
    d = cohens_d(cola[m].values, others[m].values)
    ci_low, ci_high = bootstrap_ci(cola[m].values, others[m].values)
    ci_str = f"[{ci_low:.2f}, {ci_high:.2f}]"
    excludes_zero = "*" if ci_low > 0 or ci_high < 0 else ""
    print(f"{m.replace('_resid', ''):<25} {d:>+10.2f} {ci_str:>20} {excludes_zero}")

print("-" * 60)
print("* = 95% CI excludes zero (robust effect)")

In [ ]:
# Permutation test
def permutation_test(values, labels, n_perm=1000, seed=42):
    rng = np.random.RandomState(seed)
    grammar_mask = labels == 'cola'
    observed_d = abs(cohens_d(values[grammar_mask], values[~grammar_mask]))
    
    null_ds = []
    for _ in range(n_perm):
        shuffled = rng.permutation(labels)
        shuffled_mask = shuffled == 'cola'
        null_ds.append(abs(cohens_d(values[shuffled_mask], values[~shuffled_mask])))
    
    return observed_d, np.mean(np.array(null_ds) >= observed_d)

print("Permutation Test Results (1000 shuffles)")
print("=" * 60)
print(f"{'Metric':<30} {'Observed d':>12} {'p-value':>12}")
print("-" * 60)

for m in ['mean_influence_resid', 'top_100_concentration_resid', 'n_active_resid']:
    obs_d, p_val = permutation_test(df[m].values, df['source'].values)
    sig = "***" if p_val < 0.001 else ("**" if p_val < 0.01 else ("*" if p_val < 0.05 else ""))
    print(f"{m.replace('_resid', ''):<30} {obs_d:>12.3f} {p_val:>12.4f} {sig}")

print("-" * 60)
print("*** p<0.001, ** p<0.01, * p<0.05")

---

## 6. PCA Shows Natural Clustering

Do task types naturally cluster in metric space?

In [ ]:
# Display PCA clustering figure
fig_path = FIGURES_DIR / 'pca_clustering.png'
if fig_path.exists():
    display(Image(filename=str(fig_path), width=700))
else:
    print(f"Figure not found: {fig_path}")

In [ ]:
# Compute PCA on all metrics
all_metrics = ['n_active', 'n_edges', 'mean_influence', 'max_influence', 
               'top_100_concentration', 'mean_activation', 'max_activation',
               'logit_entropy', 'max_logit_prob']

available = [m for m in all_metrics if m in df.columns]
X = df[available].values
X_std = StandardScaler().fit_transform(X)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_std)

print("PCA Variance Explained:")
print("=" * 40)
for i, var in enumerate(pca.explained_variance_ratio_, 1):
    print(f"  PC{i}: {var:.1%}")
print(f"  Total (3 PCs): {pca.explained_variance_ratio_.sum():.1%}")

In [ ]:
# Create PCA scatter plot
fig, ax = plt.subplots(figsize=(10, 8))

for src in ['cola', 'winogrande', 'snli', 'hellaswag', 'paws']:
    mask = df['source'] == src
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               alpha=0.6, label=LABELS[src], 
               color=COLORS[src], s=60,
               edgecolors='white', linewidth=0.5)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
ax.set_title('Task Types Cluster in Attribution Metric Space')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## Summary

### What We Found

1. **Different tasks produce different activation patterns** - This is real, not an artifact.

2. **The signal is in influence and concentration**, not feature counts:
   - `mean_influence`: d = 1.08 (large effect, persists after length control)
   - `concentration`: d = 0.87 (large effect, persists after length control)
   - `n_active`: d = 0.07 (collapses - was just measuring text length)

3. **Tasks form a gradient from focused to diffuse**:
   - Grammar (CoLA): High influence, high concentration = focused computation
   - Paraphrase (PAWS): Low influence, low concentration = diffuse computation

4. **Statistical validation is strong**:
   - Bootstrap 95% CIs exclude zero
   - Permutation test p < 0.001

### What This Means

Activation topology measures **how** a model computes, not **what** it computes. Different cognitive operations (grammar checking vs. reasoning) require different internal structures.